In [1]:
HOME_PATH = "/home/tiny-faces"

ROOT="data/WIDER"
TRAINDATA=ROOT + "/wider_face_split/wider_face_train_bbx_gt.txt"
VALDATA=ROOT + "/wider_face_split/wider_face_val_bbx_gt.txt"
TESTDATA=ROOT + "/wider_face_split/wider_face_test_filelist.txt"
CHECKPOINT = "weights/checkpoint_50_best.pth"
RESULT_DIR = "result"

%cd "$HOME_PATH"

/home/tiny-faces


In [2]:
!pip list | grep pyclust

pyclust                       0.2.0
pyclustering                  0.10.1.2


In [3]:
!which pip

/opt/conda/bin/pip


In [4]:
!which python

/opt/conda/bin/python


## test

In [5]:
!python evaluate.py "$TESTDATA" --dataset-root "$ROOT" --checkpoint "$CHECKPOINT" --split test

Dataset loaded
16097 samples in the test dataset
/opt/conda/lib/python3.8/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  warnings.warn(
/opt/conda/lib/python3.8/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=ResNet101_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet101_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
  0%|                                      | 28/16097 [00:13<2:12:38,  2.02it/s]
Traceback (most recent call last):
  File "evaluate.py", line 116, in <module>
    main()
  File "evaluate.py", line 110, in main
    run(model, val_loader, templates, args.prob_thresh, args.nms_thresh,
  File "evaluate.py", line 87, in run
    dets = train

In [8]:
!python evaluate.py "$VALDATA" --dataset-root "$ROOT" --checkpoint "$CHECKPOINT" --split val

Dataset loaded
3226 samples in the val dataset
/opt/conda/lib/python3.8/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  warnings.warn(
/opt/conda/lib/python3.8/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=ResNet101_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet101_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
 17%|██████▉                                 | 563/3226 [04:02<19:05,  2.32it/s]
Traceback (most recent call last):
  File "evaluate.py", line 116, in <module>
    main()
  File "evaluate.py", line 110, in main
    run(model, val_loader, templates, args.prob_thresh, args.nms_thresh,
  File "evaluate.py", line 87, in run
    dets = trainer

## visual

In [5]:
VALDATA_img = "data/WIDER/WIDER_val/images/"
TESTDATA_img = "data/WIDER/WIDER_test/images/"
target_path = "results/22--Picnic/22_Picnic_Picnic_22_985.txt"

In [6]:
import glob
import re
import os
from loguru import logger
import cv2

VALDATA_img_cat = glob.glob(VALDATA_img + "*")
TESTDATA_img_cat = glob.glob(TESTDATA_img + "*")
cat_id = 0

In [7]:
VALDATA_img_cat[cat_id]

'data/WIDER/WIDER_val/images/0--Parade'

In [8]:
VALDATA_img_list = glob.glob(VALDATA_img_cat[0] + "/*")
TESTDATA_img_list = glob.glob(TESTDATA_img_cat[0] + "/*")

In [9]:
VALDATA_img_list[:10]

['data/WIDER/WIDER_val/images/0--Parade/0_Parade_marchingband_1_1004.jpg',
 'data/WIDER/WIDER_val/images/0--Parade/0_Parade_marchingband_1_104.jpg',
 'data/WIDER/WIDER_val/images/0--Parade/0_Parade_marchingband_1_1045.jpg',
 'data/WIDER/WIDER_val/images/0--Parade/0_Parade_marchingband_1_139.jpg',
 'data/WIDER/WIDER_val/images/0--Parade/0_Parade_marchingband_1_147.jpg',
 'data/WIDER/WIDER_val/images/0--Parade/0_Parade_marchingband_1_149.jpg',
 'data/WIDER/WIDER_val/images/0--Parade/0_Parade_marchingband_1_156.jpg',
 'data/WIDER/WIDER_val/images/0--Parade/0_Parade_marchingband_1_172.jpg',
 'data/WIDER/WIDER_val/images/0--Parade/0_Parade_marchingband_1_188.jpg',
 'data/WIDER/WIDER_val/images/0--Parade/0_Parade_marchingband_1_20.jpg']

In [10]:
TESTDATA_img_list[:10]

['data/WIDER/WIDER_test/images/0--Parade/0_Parade_marchingband_1_1007.jpg',
 'data/WIDER/WIDER_test/images/0--Parade/0_Parade_marchingband_1_1008.jpg',
 'data/WIDER/WIDER_test/images/0--Parade/0_Parade_marchingband_1_1010.jpg',
 'data/WIDER/WIDER_test/images/0--Parade/0_Parade_marchingband_1_1012.jpg',
 'data/WIDER/WIDER_test/images/0--Parade/0_Parade_marchingband_1_1016.jpg',
 'data/WIDER/WIDER_test/images/0--Parade/0_Parade_marchingband_1_1019.jpg',
 'data/WIDER/WIDER_test/images/0--Parade/0_Parade_marchingband_1_102.jpg',
 'data/WIDER/WIDER_test/images/0--Parade/0_Parade_marchingband_1_1024.jpg',
 'data/WIDER/WIDER_test/images/0--Parade/0_Parade_marchingband_1_1025.jpg',
 'data/WIDER/WIDER_test/images/0--Parade/0_Parade_marchingband_1_1026.jpg']

In [11]:
for img_path in VALDATA_img_list[:10]:
    result_path = img_path.replace("data/WIDER/WIDER_val/images", "results").replace(".jpg", ".txt")
    out_path = img_path.replace("data/WIDER/WIDER_val/images", "outputs/val")
    
    out_dir = "/".join(out_path.split("/")[0:-1])
    os.makedirs(out_dir, exist_ok=True)
    
    logger.info("{:>20}:{}".format("out_path", out_path))
    logger.info("{:>20}:{}".format("out_dir", out_dir))
    is_file = os.path.isfile(result_path)

    logger.info("{:>20}:{}".format("is_file", is_file))
    logger.info("{:>20}:{}".format("result_path", result_path))
    if(is_file):
        im = cv2.imread(img_path)
        f = open(result_path, 'r')

        datalist = f.readlines()
        logger.info("{:>20}:{}".format("datalist", len(datalist)))
        for iLine in range(len(datalist)):
            datalist_split = datalist[iLine].split(" ")
            logger.info("{:>20}:{}".format("list num", len(datalist_split)))
            logger.info("{:>20}:{}".format("list ", datalist_split))
            if(len(datalist_split) == 5):
                x, y, w, h, conf = datalist_split
                conf = conf.replace("\n", "")
                if(float(conf) > 0):
                    cv2.rectangle(im, (int(x), int(y)), (int(x)+int(w), int(y)+int(h)), (255, 0, 0))
            
        cv2.imwrite(out_path, im)


2023-03-20 03:40:04.550 | INFO     | __main__:<module>:8 -             out_path:outputs/val/0--Parade/0_Parade_marchingband_1_1004.jpg
2023-03-20 03:40:04.551 | INFO     | __main__:<module>:9 -              out_dir:outputs/val/0--Parade
2023-03-20 03:40:04.552 | INFO     | __main__:<module>:12 -              is_file:True
2023-03-20 03:40:04.552 | INFO     | __main__:<module>:13 -          result_path:results/0--Parade/0_Parade_marchingband_1_1004.txt
2023-03-20 03:40:04.567 | INFO     | __main__:<module>:19 -             datalist:60
2023-03-20 03:40:04.568 | INFO     | __main__:<module>:22 -             list num:1
2023-03-20 03:40:04.568 | INFO     | __main__:<module>:23 -                list :['0_Parade_marchingband_1_1004.jpg\n']
2023-03-20 03:40:04.568 | INFO     | __main__:<module>:22 -             list num:1
2023-03-20 03:40:04.569 | INFO     | __main__:<module>:23 -                list :['58\n']
2023-03-20 03:40:04.569 | INFO     | __main__:<module>:22 -             list num:5
20

In [ ]:
for img_path in TESTDATA_img_list[:10]:
    result_path = img_path.replace("data/WIDER/WIDER_test/images", "results").replace(".jpg", ".txt")
    out_path = img_path.replace("data/WIDER/WIDER_test/images", "outputs/test")

    
    is_file = os.path.isfile(result_path)

    logger.info("{:>20}:{}".format("is_file", is_file))
    logger.info("{:>20}:{}".format("result_path", result_path))
    if(is_file):
        im = cv2.imread(img_path)
        f = open(result_path, 'r')

        datalist = f.readlines()
        logger.info("{:>20}:{}".format("datalist", len(datalist)))
        for iLine in range(len(datalist)):
            datalist_split = datalist[iLine].split(" ")
            logger.info("{:>20}:{}".format("list num", len(datalist_split)))
            logger.info("{:>20}:{}".format("list ", datalist_split))
            if(len(datalist_split) == 5):
                x, y, w, h, conf = datalist_split
                conf = conf.replace("\n", "")
                if(float(conf) > -6):
                    cv2.rectangle(im, (int(x), int(y)), (int(x)+int(w), int(y)+int(h)), (255, 0, 0))
            
        cv2.imwrite(out_path, im)


2023-03-20 03:38:14.956 | INFO     | __main__:<module>:7 -             out_path:outputs/test/0--Parade/0_Parade_marchingband_1_1007.jpg
2023-03-20 03:38:14.957 | INFO     | __main__:<module>:8 -              out_dir:outputs/test/0--Parade
2023-03-20 03:38:14.957 | INFO     | __main__:<module>:11 -              is_file:True
2023-03-20 03:38:14.958 | INFO     | __main__:<module>:12 -          result_path:results/0--Parade/0_Parade_marchingband_1_1007.txt
2023-03-20 03:38:14.969 | INFO     | __main__:<module>:18 -             datalist:33
2023-03-20 03:38:14.970 | INFO     | __main__:<module>:21 -             list num:1
2023-03-20 03:38:14.970 | INFO     | __main__:<module>:22 -                list :['0_Parade_marchingband_1_1007.jpg\n']
2023-03-20 03:38:14.970 | INFO     | __main__:<module>:21 -             list num:1
2023-03-20 03:38:14.971 | INFO     | __main__:<module>:22 -                list :['31\n']
2023-03-20 03:38:14.971 | INFO     | __main__:<module>:21 -             list num:5
